In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
echo = pd.read_csv(os.path.expanduser('~/dropbox/ecgnet-as/data/mgh-echo-lab/data.csv'))
echo = echo[["mrn", "date", "AV Peak Gradient", "AV Mean Gradient", "AV Peak Velocity", "AV Area", "source"]]
echo["mrn"] = echo["mrn"].astype(int)
echo["date"] = pd.to_datetime(echo["date"]).dt.date
echo = echo.dropna(subset=["mrn", "date"])
print(f"{len(echo)} Echos from Echo Lab")

In [ ]:
edw = pd.read_csv(os.path.expanduser('~/dropbox/ecgnet-as/data/edw/edw-echo-no-conversion.csv'))
edw = edw[['mrn', 'date', 'AV Peak Gradient', 'AV Mean Gradient', 'AV Peak Velocity', 'AV Area']]
edw['source'] = pd.Series('EDW').repeat(len(edw)).reset_index(drop=True)
edw['mrn'] = edw['mrn'].astype(int)
edw["date"] = pd.to_datetime(edw["date"]).dt.date
edw = edw.dropna(subset=["mrn", "date"])
print(f"{len(edw)} Echos from EDW")

In [ ]:
# skip patients with multiple echos on the same day
# (cannot do datetime matching because the echo lab data lacks times)
edw = edw.drop_duplicates(['mrn', 'date'], keep=False)
echo = echo.drop_duplicates(['mrn', 'date'], keep=False)

# keep Epic (from MGH Echo Lab) over EDW
both = pd.concat([echo, edw]).reset_index(drop=True).sort_values('source')
both = both.drop_duplicates(['mrn', 'date', 'AV Peak Gradient', 'AV Mean Gradient', 'AV Peak Velocity', 'AV Area'], keep='last')
assert len(both[both.duplicated(['mrn', 'date'])]) == 0

In [ ]:
both.to_csv(os.path.expanduser('~/dropbox/ecgnet-as/data/combined.csv'), index=False)